In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*/')

In [4]:
df_green.registerTempTable('green')

c:\Users\Windows\anaconda3\envs\dev\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [29]:
df_green_revenue = spark.sql("""
SELECT
    -- Reveneue grouping 
    date_trunc("hour", "pickup_datetime") AS revenue_month, 
    PULocationID AS zone,
                      
    SUM(total_amount) AS revenue_monthly_total_amount,
    COUNT(1) as number_records
FROM green

WHERE lpep_pickup_datetime >='2020-01-01 00:00:00'
GROUP BY 1,2
ORDER BY 1,2
""")

In [30]:
df_green_revenue.show()

+-------------+----+----------------------------+--------------+
|revenue_month|zone|revenue_monthly_total_amount|number_records|
+-------------+----+----------------------------+--------------+
|         null|   1|                     4176.16|            44|
|         null|   2|          251.65000000000003|             5|
|         null|   3|          127511.81999999998|          4232|
|         null|   4|          22788.830000000005|           666|
|         null|   5|          16156.439999999997|           223|
|         null|   6|           6531.390000000002|            94|
|         null|   7|          1045829.2199999562|         69053|
|         null|   8|          2375.4000000000005|            83|
|         null|   9|                    68754.06|          2327|
|         null|  10|          247243.94999999955|          7019|
|         null|  11|           76260.66999999998|          2615|
|         null|  12|                      347.37|             9|
|         null|  13|     

In [33]:
df_green_revenue.repartition(10).write.parquet('data/report/revenue_green', mode="overwrite")